In [1]:
from fasthtml.common import *

In [5]:
db = database('uplayers.db')
players = db.t.players
if players not in db.t:
    players.create(id=int, name=str, points=int, auth_method_id=int, pk='id')
    
auth_methods = db.t.auth_methods
if auth_methods not in db.t:
    auth_methods.create(id=int, name=str, pk='id')
    auth_methods.insert({'name': "Unknown"}) # id 0
    auth_methods.insert({'name': "Huggingface"}) # id 1
    auth_methods.insert({'name': "Gmail"}) # id 2
    db.add_foreign_keys((('players', 'auth_method_id', 'auth_methods', 'id'),))

db.t.players.foreign_keys

[ForeignKey(table='players', column='auth_method_id', other_table='auth_methods', other_column='id')]

In [3]:
db = database('uplayers.db')
players = db.t.players
list(players.rows)

[]

In [7]:
players.insert({'name': "dsadsa", 'points': 20, 'auth_method_id': 1})

{'id': 1, 'name': 'dsadsa', 'points': 20, 'auth_method_id': 1}

In [9]:
db.q(f"select * from {players} where {players.c.name} like 'dsadsa' limit 1")

[]

In [30]:
db.q(f"select * from {players} where {players.c.name} like 'dsadsa' limit 1")[0]['points']

20

In [4]:
players.drop()

In [10]:
list(players.rows)

[{'id': 1, 'name': 'Mihaiii', 'points': 194}]

In [9]:
players

<Table players (id, name, points)>

In [18]:
ids = ", ".join([str(x) for x in [1, 2,3]])
db.q(f"select * from {players} where {players.c.id} in ({ids})")

[{'id': 1, 'name': 'Mihaiii', 'points': 194}]

In [11]:
import sqlite3

# Connect to the database (replace 'your_database.db' with your database file)
conn = sqlite3.connect('uplayers.db')
cursor = conn.cursor()

# Query to get all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

In [15]:
tables[2][0]

'trivias'

In [10]:
trivias = db.t.trivias
if trivias not in db.t:
    trivias.create(topic=str, question=str, option_A=str, option_B=str, option_C=str, option_D=str, correct_option=str, pk='id')

In [1]:
import sqlite3

In [4]:
db = database('uplayers.db')

In [6]:
from datasets import load_dataset
trivias = db.t.trivias
if trivias not in db.t:
    #bulk import from HF dataset
    dataset = load_dataset('Mihaiii/trivia_single_choice-4-options', split='train')
    conn = sqlite3.connect('uplayers.db')
    cursor = conn.cursor()
    cursor.execute('''
    CREATE TABLE trivias (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        topic TEXT NOT NULL,
        question TEXT NOT NULL,
        option_A TEXT NOT NULL,
        option_B TEXT NOT NULL,
        option_C TEXT NOT NULL,
        option_D TEXT NOT NULL,
        correct_option TEXT NOT NULL
    );
    ''')
    conn.commit()
    insert_query = "INSERT INTO trivias (topic, question, option_A, option_B, option_C, option_D, correct_option) VALUES (?, ?, ?, ?, ?, ?, ?)"
    conn.execute('BEGIN TRANSACTION')
    for record in dataset:
        cursor.execute(insert_query, (record['topic'], record['question'], record['option_A'], record['option_B'], record['option_C'], record['option_D'], record['correct_option']))
    conn.commit()
    conn.close()

c:\Users\aprop\OneDrive\Desktop\trivia\github\trivia\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 16529/16529 [00:00<00:00, 178744.01 examples/s]


In [7]:
trivia_recs = db.q(f"SELECT * FROM {trivias} ORDER BY RANDOM() LIMIT 5")  

In [8]:
trivia_recs

[{'id': 2814,
  'topic': 'Golden Age of Video Gaming (1970s-1980s)',
  'question': 'Which iconic arcade game, released in 1980, featured a yellow character eating dots in a maze while avoiding ghosts?',
  'option_A': 'Space Invaders',
  'option_B': 'Pac-Man',
  'option_C': 'Donkey Kong',
  'option_D': 'Asteroids',
  'correct_option': 'B'},
 {'id': 8392,
  'topic': 'Astronomical Discoveries by Hubble Telescope',
  'question': 'Which of the following is one of the most famous images captured by the Hubble Space Telescope, showcasing a region of star formation in the Eagle Nebula?',
  'option_A': 'Pillars of Creation',
  'option_B': 'The Great Red Spot',
  'option_C': 'The Horsehead Nebula',
  'option_D': 'The Andromeda Galaxy',
  'correct_option': 'A'},
 {'id': 11627,
  'topic': 'Phase Transitions in Matter',
  'question': 'What is the term for the phase transition from a solid directly to a gas, skipping the liquid phase?',
  'option_A': 'Melting',
  'option_B': 'Condensation',
  'optio